Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [8]:
from random import random, seed
import numpy as np
import matplotlib.pyplot as plt
from itertools import accumulate
from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [9]:
UNIVERSE_SIZE = 10_000
NUM_SETS = 1_000
DENSITY = 0.1

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [10]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.power(SETS.sum(axis=1), 1.1)

## Helper Functions

In [11]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Continuous space optimization

In [38]:
def one_lambda_es(solution: np.ndarray, strength: float, Lambda: int) -> np.ndarray:
    CHILDREN = np.zeros((Lambda, solution.shape[0]), dtype=bool)
    
    for child in range(Lambda):
        mask = rng.random(solution.shape[0]) < strength  # Assurer que mask est de la taille de solution
        if not np.any(mask):  
            mask[np.random.randint(solution.shape[0])] = True
        CHILDREN[child] = np.logical_xor(solution, mask)
    
    return CHILDREN

## Hill climber

In [40]:
def hill_climb(universe_size, num_sets, density):

    UNIVERSE_SIZE = universe_size
    NUM_SETS = num_sets
    DENSITY = density

    rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))
    SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
    for s in range(UNIVERSE_SIZE):
        if not np.any(SETS[:, s]):
            SETS[np.random.randint(NUM_SETS), s] = True
    COSTS = np.power(SETS.sum(axis=1), 1.1)

    def valid(solution):
        """Checks whether the solution covers all elements in the universe"""
        selected_sets = SETS[solution]
        return np.all(np.logical_or.reduce(selected_sets))

    def cost(solution):
        return COSTS[solution].sum()

    def fitness(solution):
        return valid(solution), -cost(solution)

    BUFFER_SIZE = 5 #rule of one out of five
    Lambda = 10

    #Initialization
    solution = rng.random(NUM_SETS) < 1  
    solution_fitness = fitness(solution)[1]
    history = [float(solution_fitness)]
    fitsness1 = (fitness(solution))

    #"Simulated Annealing" 
    strength = 0.5
    buffer = list()

    for steps in range (10_000):
        #using one_lambda_method
        new_solutions = one_lambda_es(solution, strength, Lambda)

        best_fitness=solution_fitness 
        new_solution=solution  

        for i in range(Lambda):
            valid_solution, f = fitness(new_solutions[i]) 
            if valid_solution and f > best_fitness: 
                best_fitness = f  
                new_solution = new_solutions[i]  

        history.append(float(best_fitness))

        #1 out of 5 method
        buffer.append(best_fitness > solution_fitness)
        buffer = buffer[-BUFFER_SIZE:]  

        if sum(buffer) > 1:  
            strength *= 1.1
        elif sum(buffer) == 0:  
            strength /= 1.1

        if best_fitness > solution_fitness:
            solution = new_solution
            solution_fitness = best_fitness

    fitness2 = (fitness(solution))
    return (fitsness1,fitness2)


## Results

In [ ]:
#funtion structure idea (list) found on internet
def run_experiments():
    test_cases = [
        (100, 10, 0.2),
        (1_000, 100, 0.2),
        (10_000, 1_000, 0.2),
        (100_000, 10_000, 0.1),
        (100_000, 10_000, 0.2),
        (100_000, 10_000, 0.3)]

    for idx, (universe_size, num_sets, density) in enumerate(test_cases, 1):
        print(f"\nTest Case {idx}:Universe Size={universe_size}, Num Sets={num_sets}, Density={density}")
        fitness_before, fitness_after = hill_climb(universe_size, num_sets, density)
        
        print(f"Fitness Before: {fitness_before}")
        print(f"Fitness After: {fitness_after}")

run_experiments()



Test Case 1: Universe Size = 100, Num Sets = 10, Density = 0.2
Fitness Before: (np.True_, np.float64(-309.1213513636662))
Fitness After: (np.True_, np.float64(-286.55331289056136))

Test Case 2: Universe Size = 1000, Num Sets = 100, Density = 0.2
Fitness Before: (np.True_, np.float64(-33820.9278750089))
Fitness After: (np.True_, np.float64(-7256.735508794502))

Test Case 3: Universe Size = 10000, Num Sets = 1000, Density = 0.2
Fitness Before: (np.True_, np.float64(-4272918.687624525))
Fitness After: (np.True_, np.float64(-134130.7120029037))

Test Case 4: Universe Size = 100000, Num Sets = 10000, Density = 0.1
